In [117]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
import pandas as pd

load_dotenv()

llm = ChatOpenAI(model="gpt-5",temperature=0)

In [118]:
def extract_dataframe_from_result(intermediate_steps):
    """
    Extract the dataframe result from agent intermediate steps.
    Returns the actual pandas DataFrame object when possible.
    """
    if not intermediate_steps:
        return None
    
    # Get the result from the last tool execution
    last_result = intermediate_steps[-1][1]
    
    # If it's already a DataFrame, return it
    if isinstance(last_result, pd.DataFrame):
        return last_result
    
    # If it's a string representation, try to extract the query and re-execute
    action = intermediate_steps[-1][0]
    if hasattr(action, 'tool_input') and 'query' in action.tool_input:
        query = action.tool_input['query']
        print(f"Executing query: {query}")
        try:
            result_df = eval(query.split('\n')[-1])
            if isinstance(result_df, pd.DataFrame):
                return result_df
        except:
            pass
    return last_result

In [119]:
def analysis_agent(path, llm, query):
    df = pd.read_csv(path)
    agent_executor = create_pandas_dataframe_agent(
        llm,
        df,
        agent_type="tool-calling",
        allow_dangerous_code=True,
        return_intermediate_steps=True  # Add this line
    )

    result = agent_executor.invoke({"input": query})
    
    # Check if intermediate_steps exists before trying to extract
    if "intermediate_steps" in result:
        df = extract_dataframe_from_result(result["intermediate_steps"])
    else:
        df = None
        print("No intermediate steps found in result")
    
    return result, df

In [120]:
response,df = analysis_agent("C:\\vscode\\csv_agent\\data\\data.csv", llm, "what is the profit margin for each product category")

In [121]:
df

,product_category,revenue,cogs,profit,profit_margin_pct
1,Professional Services,74376168.95,36861690.53,37514478.42,50.44
2,Software Subscription,36350192.23,18171125.67,18179066.56,50.01
3,Training,8917121.27,4512474.23,4404647.04,49.40
0,Hardware,69663392.19,35360408.29,34302983.90,49.24
